### Building evasion vs. mean time interval : by Bus by Service - THIS IS A TEST

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from importlib import reload
import datetime as dt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
from Utils import TransantiagoConstants

In [3]:
first_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/1st_quarter.xlsx"
second_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/2nd_quarter.xlsx"
third_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/3rd_quarter.xlsx"
codes_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/04_DTPM/codes_services.xlsx"

In [4]:
first_quarter_evasion = pd.read_excel(first_quarter_path, encoding = 'latin-1')
second_quarter_evasion = pd.read_excel(second_quarter_path, encoding = 'latin-1')
third_quarter_evasion = pd.read_excel(third_quarter_path, encoding = 'latin-1')
codes = pd.read_excel(codes_path, encoding = 'latin-1')

In [5]:
first_quarter_evasion['TIEMPO'] = first_quarter_evasion['HORA'].astype(str)+':'+first_quarter_evasion['MINUTOS'].astype(str)+':00'
second_quarter_evasion['TIEMPO'] = second_quarter_evasion['HORA'].astype(str)+':'+second_quarter_evasion['MINUTOS'].astype(str)+':00'
third_quarter_evasion['TIEMPO'] = third_quarter_evasion['HORA'].astype(str)+':'+third_quarter_evasion['MINUTOS'].astype(str)+':00'

* Consider to change this using the rename function of pandas

In [6]:
first_quarter_evasion.columns = ['FECHA', 'SERVICIO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
second_quarter_evasion.columns = ['FECHA', 'SERVICIO','TIPO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
third_quarter_evasion.columns = ['FECHA', 'SERVICIO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
del second_quarter_evasion['TIPO']

In [7]:
type(first_quarter_evasion.loc[0,'TIEMPO'])

str

In [8]:
type(first_quarter_evasion.loc[0,'FECHA'])

pandas._libs.tslib.Timestamp

In [9]:
frames = [first_quarter_evasion, second_quarter_evasion, third_quarter_evasion]
evasion = pd.concat(frames, keys=['first', 'second', 'third'])

In [10]:
evasion.loc[:,'SERVICIO'] = evasion.loc[:,'SERVICIO'].apply(str)

* Processing codes before merging. Merge should be made only by codes_ida or codes_ret. Be aware of it

In [11]:
codes_ida = codes[codes['DIRECTION']=='Ida']
codes_ret = codes[codes['DIRECTION']=='Ret']

# Both, USER_CODE and TS_CODE should be strings, in both codes_ida and codes_ret.
codes_ida.loc[:,'USER_CODE'] = codes_ida.loc[:,'USER_CODE'].apply(str)
codes_ida.loc[:,'TS_CODE'] = codes_ida.loc[:,'TS_CODE'].apply(str)

codes_ret.loc[:,'USER_CODE'] = codes_ret.loc[:,'USER_CODE'].apply(str)
codes_ret.loc[:,'TS_CODE'] = codes_ret.loc[:,'TS_CODE'].apply(str)

del codes_ida['DIRECTION']
del codes_ret['DIRECTION']


#Cleaning codes from special services. Be aware of this since it might generates errors in the final merging with trx database.
clean_codes_ida = codes_ida[(~codes_ida["TS_CODE"].str.contains('y'))&(~codes_ida["TS_CODE"].str.contains('Y'))]
clean_codes_ret = codes_ida[(~codes_ida["TS_CODE"].str.contains('y'))&(~codes_ida["TS_CODE"].str.contains('Y'))]

clean_codes_ida = clean_codes_ida.rename(columns = {'USER_CODE':'SERVICIO'})
clean_codes_ret = clean_codes_ret.rename(columns = {'USER_CODE':'SERVICIO'})

C:\Users\leoca_000\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
evasion = pd.merge(evasion,clean_codes_ida, on=['SERVICIO'], how='left')

In [13]:
evasion.loc[evasion['TS_CODE'].isnull(),'SERVICIO'].unique()

array(['D06'], dtype=object)

* THIS IS HARDCODED

In [14]:
evasion.loc[evasion['TS_CODE'].isnull(),'TS_CODE']='446'

In [15]:
evasion.head()

,FECHA,SERVICIO,PATENTE,PUERTAS,N_PUERTA,LUGAR_INICIO,HORA_INICIO,HORA,MINUTO,INGRESAN,NO_VALIDAN,TP,TIEMPO,UN,TS_CODE
0,2017-01-10,214,BJFH98,3,1,AV. ADOLFO LOPEZ MATEO,07:10:00,7,10,15,1,P,7:10:00,2.0,214
1,2017-01-10,214,BJFH98,3,1,AV. ADOLFO LOPEZ MATEO,07:10:00,7,12,9,0,P,7:12:00,2.0,214
2,2017-01-10,214,BJFH98,3,1,AV. ADOLFO LOPEZ MATEO,07:10:00,7,13,6,2,P,7:13:00,2.0,214
3,2017-01-10,214,BJFH98,3,1,AV. ADOLFO LOPEZ MATEO,07:10:00,7,18,8,1,P,7:18:00,2.0,214
4,2017-01-10,214,BJFH98,3,1,AV. ADOLFO LOPEZ MATEO,07:10:00,7,20,5,1,P,7:20:00,2.0,214


* Processing evasion-ddbb before merging

In [16]:
evasion.loc[:,'PATENTE'] =  evasion.loc[:,'PATENTE'].str.replace(' ','')

evasion.loc[:,'TS_CODE'] = evasion.loc[:,'TS_CODE'].apply(str)

evasion = evasion.rename(columns = {'SERVICIO':'SERVICIO_USUARIO', 'TS_CODE':'SERVICIO'})

evasion_paradero = evasion[evasion['TP']=='P']
evasion_paradero_first = evasion_paradero[evasion_paradero['N_PUERTA']==1]

In [17]:
type(evasion['SERVICIO'][0])

str

* Getting evasion by bus by service

In [18]:
f = {'SERVICIO':['count'],'INGRESAN':['sum'], 'NO_VALIDAN':['sum']}

summary = evasion_paradero_first.groupby(['FECHA','PATENTE','SERVICIO']).agg(f)
summary.columns=['EVASION_COUNT', 'TOTAL_INGRESAN', 'TOTAL_NO_VALIDAN']
summary.reset_index(inplace=True)

In [19]:
summary.head()

,FECHA,PATENTE,SERVICIO,EVASION_COUNT,TOTAL_INGRESAN,TOTAL_NO_VALIDAN
0,2017-01-10,BJFH78,214,23,96,17
1,2017-01-10,BJFH98,214,21,117,17
2,2017-01-10,BJFK40,214,17,40,0
3,2017-01-10,FLXC94,214,29,78,26
4,2017-01-10,FLXC98,214,23,29,13


* Building complete etapas ddbb.

In [20]:
common_dates = TransantiagoConstants.common_dates

In [21]:
reduced_etapas = pd.DataFrame()

for date in common_dates:
    base_input_path = 'C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/01_SSH/02_TIMESBYBUSBYSERVICE/01_ORIGINAL/'
    file_name = date + '-meanByBusByService.csv'
    grouped_by_bus = pd.read_csv(base_input_path+file_name, encoding = 'latin-1',sep =";")
    reduced_etapas = pd.concat([reduced_etapas,grouped_by_bus])


* Processing etapas ddbb before merging

In [22]:
del reduced_etapas['Unnamed: 0']
reduced_etapas = reduced_etapas.rename(columns = {'fecha':'FECHA', 'sitio_subida':'PATENTE', 'count': 'ETAPAS_COUNT', 'servicio_subida' : 'SERVICIO'})
reduced_etapas['FECHA'] = pd.to_datetime(reduced_etapas.FECHA)
reduced_etapas['PATENTE'] = reduced_etapas['PATENTE'].str.replace("-", "")
reduced_etapas['PATENTE'] = reduced_etapas['PATENTE'].str.replace(" ", "")

In [23]:
reduced_etapas.reset_index(drop=True,inplace=True)

In [24]:
type(reduced_etapas.loc[0,'FECHA'])

pandas._libs.tslib.Timestamp

## <strong>TEST MERGING</strong>

1 - BASICS 

In [25]:
evasion_vs_mean = pd.merge(summary,reduced_etapas, on=['PATENTE','SERVICIO','FECHA'], how='left')

In [26]:
evasion_vs_mean_length = len(evasion_vs_mean.index)
print('The total number of rows in evasion vs. mean ddbb is: ' +str(evasion_vs_mean_length))

The total number of rows in evasion vs. mean ddbb is: 1184


* Getting rid of missing values

In [27]:
clean_evasion_vs_mean = evasion_vs_mean[evasion_vs_mean['mean']>0]

In [28]:
clean_evasion_vs_mean_length = len(clean_evasion_vs_mean.index)
print('The total number of rows matching in evasion vs. mean ddbb is: ' + str(clean_evasion_vs_mean_length))
print('The percentage of matching is: ' + str(clean_evasion_vs_mean_length/evasion_vs_mean_length*100) + '%')

The total number of rows matching in evasion vs. mean ddbb is: 482
The percentage of matching is: 40.70945945945946%


= END BASICS =

2 - NUMBER OF OBSERVATIONS MISMATCHING

In [29]:
total_mismatching = evasion_vs_mean_length - clean_evasion_vs_mean_length
print('The total number of rows mismatching in evasion vs. mean ddbb is: ' + str(total_mismatching))

The total number of rows mismatching in evasion vs. mean ddbb is: 702


= END NUMBER OF OBSERVATIONS MISMATCHING =

3 - CHECKING CONDITIONS

In [30]:
within_dates_evasion_vs_mean = evasion_vs_mean[(evasion_vs_mean['FECHA']>=pd.to_datetime('2017-03-09'))&(evasion_vs_mean['FECHA']<=pd.to_datetime('2017-08-24'))]

In [31]:
within_dates_evasion_vs_mean_length = len(within_dates_evasion_vs_mean.index)
print('The number of rows within common dates range is: ' + str(within_dates_evasion_vs_mean_length))

The number of rows within common dates range is: 778


In [32]:
common_dates_timestamp = [pd.to_datetime(x) for x in common_dates]

In [33]:
common_dates_evasion_vs_mean = within_dates_evasion_vs_mean[within_dates_evasion_vs_mean['FECHA'].isin(common_dates_timestamp)]

In [34]:
common_dates_evasion_vs_mean_length = len(common_dates_evasion_vs_mean.index)
print('The total number of rows within common dates range and in common dates is: ' + str(common_dates_evasion_vs_mean_length))

The total number of rows within common dates range and in common dates is: 513


In [35]:
print('IN SUMMARY, THE ORIGINAL NUMBER OF ROWS IS ' + str(evasion_vs_mean_length) + '. FROM THIS, ' + 
     str(within_dates_evasion_vs_mean_length) + ' ARE WITHIN COMMON DATES RANGE. FROM THE LATTER, ' + 
     str(common_dates_evasion_vs_mean_length) + ' ARE WITHIN COMMON DATES RANGE AND IN FACT CORRESPOND TO A COMMON DATE. FINALLY THERE ARE ' +
     str(common_dates_evasion_vs_mean_length-clean_evasion_vs_mean_length) + ' ROWS MISMATCHING THAT SHOULD BE CHECKED IN REDUCED_ETAPAS AND SIMPLIFIED_ETAPAS FILES') 

IN SUMMARY, THE ORIGINAL NUMBER OF ROWS IS 1184. FROM THIS, 778 ARE WITHIN COMMON DATES RANGE. FROM THE LATTER, 513 ARE WITHIN COMMON DATES RANGE AND IN FACT CORRESPOND TO A COMMON DATE. FINALLY THERE ARE 31 ROWS MISMATCHING THAT SHOULD BE CHECKED IN REDUCED_ETAPAS AND SIMPLIFIED_ETAPAS FILES


= END CHECKING CONDITIONS =

4 - CHECKING PROBLEMATIC ROWS AND LOOKING INTO REDUCED_ETAPAS SIMPLIFIED_ETAPAS FILES

In [36]:
common_dates_evasion_vs_mean[common_dates_evasion_vs_mean['sum'].notnull()].head(4)

,FECHA,PATENTE,SERVICIO,EVASION_COUNT,TOTAL_INGRESAN,TOTAL_NO_VALIDAN,ETAPAS_COUNT,sum,mean,fecha_instalacion
302,2017-03-09,BKWK90,F05,49,94,27,222.0,1324.0,5.963964,NaN
303,2017-03-09,CJJH75,B07,48,125,58,192.0,1449.0,7.546875,2017-07-03
304,2017-03-09,CJKC18,B07,38,48,7,37.0,136.0,3.675676,2017-07-05
305,2017-03-09,CJRG61,F05,25,33,18,22.0,95.0,4.318182,NaN


In [37]:
common_dates_evasion_vs_mean[common_dates_evasion_vs_mean['sum'].isnull()].head(4)

,FECHA,PATENTE,SERVICIO,EVASION_COUNT,TOTAL_INGRESAN,TOTAL_NO_VALIDAN,ETAPAS_COUNT,sum,mean,fecha_instalacion
315,2017-03-14,ZB6117,125,16,14,9,NaN,NaN,NaN,NaN
333,2017-03-15,BJFJ35,363,17,43,25,NaN,NaN,NaN,NaN
334,2017-03-15,BJFJ45,363,14,22,6,NaN,NaN,NaN,NaN
352,2017-03-15,CYRY63,508,24,120,58,NaN,NaN,NaN,NaN


* Checking in reduced_etapas ...
* Checking for PATENTE == BBKB-12, SERVICIO == F11, FECHA == 2017-06-14	> Doesn't exist
* Checking for PATENTE == BFHB-25, SERVICIO == 212, FECHA == 2017-06-14 > Doesn't exist
* Checking for PATENTE == BGTL-44, SERVICIO == 334, FECHA == 2017-08-17 > Doesn't exist
* Checking for PATENTE == BGTL-77, SERVICIO == 339, FECHA == 2017-06-14 > Doesn't exist

* <font color='green'><strong>Merge is correct.</strong></font> These 35 rows should be checked in simplified_etapas files

In [38]:
from RunSilentlyDailyEtapasBuilder import RunSilentlyDailyEtapasBuilderClass

In [39]:
date = '2017-06-14'
etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
filtered_df = etapas_builder.runLightCompleteProcess()

Not found in turnstile database: 517700


In [40]:
filtered_df[filtered_df['sitio_subida']=='BBKB-12']

,index,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion,mismo_paradero,misma_patente,mismo_servicio,diferencia_tiempo,diferencia_tiempo_secs,si_torniquete,si_2017_torniquete
153,405460,2036648403,3,BUS,2017-06-14 22:27:10,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:02,2.0,False,False
154,293942,930125355,2,BUS,2017-06-14 22:27:13,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:03,3.0,False,False
155,465616,2483164525,2,BUS,2017-06-14 22:27:17,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:04,4.0,False,False
156,78158,52372738,3,BUS,2017-06-14 22:27:20,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:03,3.0,False,False
157,193185,424007954,3,BUS,2017-06-14 22:27:23,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:03,3.0,False,False
158,23227,31833762,3,BUS,2017-06-14 22:27:26,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:03,3.0,False,False
159,224809,473063026,3,BUS,2017-06-14 22:27:28,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:02,2.0,False,False
160,359613,1524637928,2,BUS,2017-06-14 22:27:28,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:00,0.0,False,False


In [41]:
filtered_df[filtered_df['sitio_subida']=='BFHB-25']

,index,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion,mismo_paradero,misma_patente,mismo_servicio,diferencia_tiempo,diferencia_tiempo_secs,si_torniquete,si_2017_torniquete


In [42]:
filtered_df[filtered_df['sitio_subida']=='BGTL-77']

,index,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion,mismo_paradero,misma_patente,mismo_servicio,diferencia_tiempo,diferencia_tiempo_secs,si_torniquete,si_2017_torniquete


In [43]:
filtered_df[filtered_df['sitio_subida']=='BBKB-13']

,index,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion,mismo_paradero,misma_patente,mismo_servicio,diferencia_tiempo,diferencia_tiempo_secs,si_torniquete,si_2017_torniquete
163,63592,47993730,2,BUS,2017-06-14 21:43:31,F11 00R,E-34-270-NS-70,BBKB-13,NaT,True,True,True,00:00:22,22.0,False,False


* Checking in simplified_etapas ...
* Checking for PATENTE == BBKB-12, SERVICIO == F11, FECHA == 2017-06-14 > Doesn't exist
* Checking for PATENTE == BFHB-25, SERVICIO == 212, FECHA == 2017-06-14 > Doesn't exist
* Checking for PATENTE == BGTL-77, SERVICIO == 339, FECHA == 2017-06-14 > Doesn't exist
* Checking for PATENTE == BBKB-13, SERVICIO == F11, FECHA == 2017-06-14 > It exists
* <font color='green'><strong>Merge is correct. TEST PASSED</strong></font>

In [44]:
date = '2017-08-17'
etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
filtered_df = etapas_builder.runLightCompleteProcess()

Not found in turnstile database: 1873694


In [45]:
filtered_df[filtered_df['sitio_subida']=='BGTL-44']

,index,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion,mismo_paradero,misma_patente,mismo_servicio,diferencia_tiempo,diferencia_tiempo_secs,si_torniquete,si_2017_torniquete


* Checking in simplified_etapas ...
* Checking for PATENTE == BGTL-44, SERVICIO == 334, FECHA == 2017-08-17 > Doesn't exist
* <font color='green'><strong>Merge is correct. TEST PASSED</strong></font>

* Checking for obs. with mean > 40 secs

In [46]:
common_dates_evasion_vs_mean[common_dates_evasion_vs_mean['mean']>40]

,FECHA,PATENTE,SERVICIO,EVASION_COUNT,TOTAL_INGRESAN,TOTAL_NO_VALIDAN,ETAPAS_COUNT,sum,mean,fecha_instalacion
917,2017-07-19,CJRZ54,457,7,6,0,105.0,5448.0,51.885714,NaN


* Checking for obs with mean > 8 secs

In [47]:
common_dates_evasion_vs_mean[common_dates_evasion_vs_mean['mean']>8]

,FECHA,PATENTE,SERVICIO,EVASION_COUNT,TOTAL_INGRESAN,TOTAL_NO_VALIDAN,ETAPAS_COUNT,sum,mean,fecha_instalacion
307,2017-03-09,FLXB83,B07,19,32,8,109.0,1222.0,11.211009,2016-07-26
337,2017-03-15,BJFR91,363,39,71,24,29.0,310.0,10.689655,NaN
338,2017-03-15,BJFR93,363,27,51,27,34.0,303.0,8.911765,NaN
346,2017-03-15,CJRT79,454,26,32,6,189.0,2021.0,10.693122,NaN
347,2017-03-15,CJRV13,454,28,83,11,290.0,4051.0,13.968966,NaN
348,2017-03-15,CJRV30,454,36,83,11,272.0,2538.0,9.330882,NaN
350,2017-03-15,CJRX81,454,10,25,4,314.0,3770.0,12.006369,NaN
351,2017-03-15,CJRZ49,454,47,118,10,328.0,3119.0,9.509146,NaN
362,2017-03-16,BBKB13,F11,44,136,18,155.0,1443.0,9.309677,NaN
363,2017-03-16,BBKD29,F11,13,8,0,79.0,639.0,8.088608,NaN


In [48]:
common_dates_evasion_vs_mean.loc[:,'EVASION_RATE'] = common_dates_evasion_vs_mean.loc[:,'TOTAL_NO_VALIDAN'] / common_dates_evasion_vs_mean.loc[:,'TOTAL_INGRESAN']

C:\Users\leoca_000\Anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\leoca_000\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [49]:
common_dates_evasion_vs_mean[common_dates_evasion_vs_mean['EVASION_RATE']>0.8]

,FECHA,PATENTE,SERVICIO,EVASION_COUNT,TOTAL_INGRESAN,TOTAL_NO_VALIDAN,ETAPAS_COUNT,sum,mean,fecha_instalacion,EVASION_RATE
530,2017-04-18,CJRZ93,452,16,21,21,NaN,NaN,NaN,NaN,1.00000
1021,2017-08-17,CJRG41,F18,28,71,60,5.0,34.0,6.8,NaN,0.84507


### Closed